In [22]:
import torch
import torch.nn as nn
import torch.optim as optim
# from simpletransformers.classification import ClassificationModel
# from simpletransformers.classification import MultiLabelClassificationModel
import pandas as pd
import logging
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score
import ast
import numpy as np
import re

logging.basicConfig(level=logging.INFO)
transformers_logger = logging.getLogger("transformers")
transformers_logger.setLevel(logging.WARNING)

from google.colab import files
uploaded = files.upload()

Saving threshold_classifier_data.csv to threshold_classifier_data.csv


In [48]:
multi_label_classification = pd.read_csv("/content/threshold_classifier_data.csv", usecols=["text", "label"])
multi_label_classification.head()

,text,label
0,The districts of Guidan-Roumji in Niger and Do...,"[1, 0, 0]"
1,"In Burkina Faso, two districts are in alert at...","[0, 0, 1]"
2,Niger: Distirit of Madarounfa is in the epide...,"[0, 0, 1]"
3,Mali: Kadiolo district is in alert phase with...,"[0, 0, 1]"
4,Spatial analysis shows an epidemic area common...,"[1, 0, 0]"


In [57]:
# summary_state = summary_state.drop(columns=["State", "Count"])
# summary_state = summary_state.rename(columns={"Summary":"text", "Label":"labels"})
# summary_state = summary_state.reset_index(drop=True)

In [25]:
separation = int(0.8 * len(multi_label_classification))
train_data = multi_label_classification[:separation]
test_data = multi_label_classification[separation:]
print(len(test_data))
train_data

57


,index,text,label
0,0,The districts of Guidan-Roumji in Niger and Do...,"[1, 0, 0]"
1,1,"In Burkina Faso, two districts are in alert at...","[0, 0, 1]"
2,2,Niger: Distirit of Madarounfa is in the epide...,"[0, 0, 1]"
3,3,Mali: Kadiolo district is in alert phase with...,"[0, 0, 1]"
4,4,Spatial analysis shows an epidemic area common...,"[1, 0, 0]"
...,...,...,...
220,220,"At week 6 of year 2018, two districts in Ghana...","[0, 1, 0]"
221,221,"At week 7 of year 2018, one district in Ghana ...","[0, 0, 1]"
222,222,A suspected meningitis outbreak has been noti...,"[0, 0, 1]"
223,223,"At week 8 of year 2018, two districts, one in ...","[0, 1, 0]"


In [6]:
pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-uc3h4635
  Running command git clone -q https://github.com/huggingface/transformers /tmp/pip-req-build-uc3h4635
     |████████████████████████████████| 3.0MB 8.2MB/s 
     |████████████████████████████████| 1.1MB 50.3MB/s 
     |████████████████████████████████| 890kB 54.9MB/s 
  Created wheel for transformers: filename=transformers-2.11.0-cp36-none-any.whl size=695686 sha256=b9d06346b56e92a3d93534f7ef33cf47bc04c15c6056ec6a80da4a6f3c6bc8a8
  Stored in directory: /tmp/pip-ephem-wheel-cache-xa_wm7y3/wheels/70/d3/52/b3fa4f8b8ef04167ac62e5bb2accb62ae764db2a378247490e
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=bbb5402949ab343073bd7f5d8bfe9c0c3b8426022607a701b52bc81e596c922a
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built transformers sacremoses


In [ ]:
#This code does language modeling as the first initial step of fine-tuning on a cusotm WHO report of meningitis
#This is so the the model can learn some of the domain specific knowledge necessary to understand some of the indivial countries' reports
!python3 "run_language_modeling.py"  \
    --output_dir=output \
    --model_type=longformer \
    --model_name_or_path="allenai/longformer-base-4096" \
    --per_device_train_batch_size=1\
    --do_train \
    --train_data_file="finetune.txt" \

usage: run_language_modeling.py [-h] [--model_name_or_path MODEL_NAME_OR_PATH]
                                [--model_type MODEL_TYPE]
                                [--config_name CONFIG_NAME]
                                [--tokenizer_name TOKENIZER_NAME]
                                [--cache_dir CACHE_DIR]
                                [--train_data_file TRAIN_DATA_FILE]
                                [--eval_data_file EVAL_DATA_FILE]
                                [--line_by_line] [--mlm]
                                [--mlm_probability MLM_PROBABILITY]
                                [--block_size BLOCK_SIZE] [--overwrite_cache]
                                --output_dir OUTPUT_DIR
                                [--overwrite_output_dir] [--do_train]
                                [--do_eval] [--do_predict]
                                [--evaluate_during_training]
                                [--per_device_train_batch_size PER_DEVICE_TRAIN_BATCH_SIZE]
      

In [7]:
device = torch.device("cuda")
from transformers import LongformerModel, LongformerTokenizer, LongformerConfig

In [8]:
tokenizer = LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
model = LongformerModel.from_pretrained('allenai/longformer-base-4096')
model.cuda()

INFO:filelock:Lock 140052896747192 acquired on /root/.cache/torch/transformers/1ae1f5b6e2b22b25ccc04c000bb79ca847aa226d0761536b011cf7e5868f0655.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b.lock


INFO:filelock:Lock 140052896747192 released on /root/.cache/torch/transformers/1ae1f5b6e2b22b25ccc04c000bb79ca847aa226d0761536b011cf7e5868f0655.ef00af9e673c7160b4d41cfda1f48c5f4cba57d5142754525572a846a1ab1b9b.lock


INFO:filelock:Lock 140055389770416 acquired on /root/.cache/torch/transformers/f8f83199a6270d582d6245dc100e99c4155de81c9745c6248077018fe01abcfb.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock


INFO:filelock:Lock 140055389770416 released on /root/.cache/torch/transformers/f8f83199a6270d582d6245dc100e99c4155de81c9745c6248077018fe01abcfb.70bec105b4158ed9a1747fea67a43f5dee97855c64d62b6ec3742f4cfdb5feda.lock


INFO:filelock:Lock 140052873793832 acquired on /root/.cache/torch/transformers/ea6da73c9ecf0c07b29510e423dfd58f1b8b59cba3c89383189ec8b0b6d00e82.4859554090880d98c2ceab35aa57afd197958c9c3b161bbfe07aeefb84557e54.lock


INFO:filelock:Lock 140052873793832 released on /root/.cache/torch/transformers/ea6da73c9ecf0c07b29510e423dfd58f1b8b59cba3c89383189ec8b0b6d00e82.4859554090880d98c2ceab35aa57afd197958c9c3b161bbfe07aeefb84557e54.lock


INFO:filelock:Lock 140052896746912 acquired on /root/.cache/torch/transformers/dfc92dbbf5c555abf807425ebdb22b55de7a17e21fe1c48cbaa5764982c1d9c0.cd65234711d2e83d420aa696eb9186cdec6ab79ef8bf090b442cf249443dfa92.lock


INFO:filelock:Lock 140052896746912 released on /root/.cache/torch/transformers/dfc92dbbf5c555abf807425ebdb22b55de7a17e21fe1c48cbaa5764982c1d9c0.cd65234711d2e83d420aa696eb9186cdec6ab79ef8bf090b442cf249443dfa92.lock


LongformerModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(4098, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): LongformerSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (query_global): Linear(in_features=768, out_features=768, bias=True)
            (key_global): Linear(in_features=768, out_features=768, bias=True)
            (value_global): Linear(in_features=768, out_features=768, bias=True)
          )
          (output): BertSelfOutpu

In [ ]:
torch.cuda.empty_cache()
#note when encoded the start of every token is added with an <s> for classification purposes
input_ids = tokenizer.encode("Hello, my dog is cute")  # Batch size 1
print(tokenizer.decode(input_ids))
print(tokenizer.decode(input_ids[0]))

<s> Hello, my dog is cute</s>
<s>


In [31]:
tokenized = multi_label_classification["text"]\
                                    .apply(lambda x: tokenizer.encode(x, add_special_tokens_tokens=True))
max_len = 0
for i in tokenized.values:
    if len(i) > max_len:
        max_len = len(i)
print(max_len)
#padded according to max_length so we can feed the model in batches - padding works as such, add 0s after the length
#of the original sequence l such that the new length is = max_len = 2530
padded = np.array([i + [0]*(max_len-len(i)) for i in tokenized.values])


627


In [27]:
np.array(padded.shape)
#Add masking so that know BERT knows where to attend to => so it can ignore padding
#adds a 1 everwhere where the condition of padded != 0 is satisfied, in other words everywhere there isn't padding
attention_mask = np.where(padded != 0, 1, 0)
#add global attention represented with 2 - for now just add 1 at the classification <s> token
attention_mask[:, [0, -1]] = 2
print(attention_mask)
device = torch.device("cuda")

input_ids = torch.tensor(padded).to(device)
attention_mask = torch.tensor(attention_mask).to(device)

def create_save_feature_representation():
  first_time = True
  for i in range(0,len(input_ids),2):
    with torch.no_grad():
      predictions, hidden_states = model(input_ids[i:i+2], attention_mask=attention_mask[i:i+2])
      scores = predictions[:,0,:].cpu().numpy()
      if not first_time:
        curr = np.load("lang_features.npy")
        scores = np.concatenate((curr, scores))
      else:
        first_time = False
      np.save("lang_features.npy",scores)

create_save_feature_representation()

[[2 1 1 ... 0 0 2]
 [2 1 1 ... 0 0 2]
 [2 1 1 ... 0 0 2]
 ...
 [2 1 1 ... 0 0 2]
 [2 1 1 ... 0 0 2]
 [2 1 1 ... 0 0 2]]


In [58]:
features = np.load("lang_features.npy")
print(multi_label_classification.iloc[0]["label"])
#convert the labels into numpy arrays since they are stored as string representations
multi_label_classification["label"] = multi_label_classification["label"].apply(lambda x: np.fromstring(x[1:-1], dtype=np.float32, sep=','))

[1, 0, 0]


In [59]:
train_features, test_features = features[:separation], features[separation:]
hidden_size = features.shape[1]

#convert to float datatype that can be then cast to a tensor in PyTorch
labels = np.array([label for label in np.array(multi_label_classification["label"].values)])
train_labels, test_labels = labels[:separation], labels[separation:]
multi_label_classification.iloc[0]["label"].shape

(3,)

In [87]:
print(train_labels.dtype)
epochs = 50
neural_network = nn.Sequential(
                    nn.Linear(hidden_size, 1000),
                    nn.ReLU(),
                    nn.BatchNorm1d(1000),
                    nn.Linear(1000, 800),
                    nn.ReLU(),
                    nn.BatchNorm1d(800),
                    nn.Linear(800, 800),
                    nn.ReLU(),
                    nn.BatchNorm1d(800),
                    nn.Linear(800, 600),
                    nn.ReLU(),
                    nn.BatchNorm1d(600),
                    nn.Linear(600, 400),
                    nn.ReLU(),
                    nn.BatchNorm1d(400),
                    nn.Linear(400, 400),
                    nn.ReLU(),
                    nn.BatchNorm1d(400),
                    nn.Linear(400, 200),
                    nn.ReLU(),
                    nn.BatchNorm1d(200),
                    nn.Linear(200, 3),
                    nn.Sigmoid(),
                    )
neural_network.cuda()
dtype = torch.cuda.FloatTensor
neural_network.type = torch.cuda.FloatTensor
train_features, test_features = torch.tensor(train_features).to(device), torch.tensor(test_features).to(device)
train_labels, test_labels = torch.tensor(train_labels).to(device), torch.tensor(test_labels).to(device)
criterion = nn.MultiLabelSoftMarginLoss()
optimizer = optim.Adam(neural_network.parameters(), lr=0.0001)

for epoch in range(epochs):
  optimizer.zero_grad()
  output = neural_network(train_features)
  loss = criterion(output, train_labels)
  print("Epoch {0} with training loss: {1}".format(epoch, loss))
  loss.backward()
  optimizer.step()    

torch.float32
Epoch 0 with training loss: 0.8055498600006104
Epoch 1 with training loss: 0.7492516040802002
Epoch 2 with training loss: 0.7172379493713379
Epoch 3 with training loss: 0.6966211199760437
Epoch 4 with training loss: 0.6818656325340271
Epoch 5 with training loss: 0.6709755659103394
Epoch 6 with training loss: 0.6625085473060608
Epoch 7 with training loss: 0.6556110978126526
Epoch 8 with training loss: 0.6498163938522339
Epoch 9 with training loss: 0.6449871063232422
Epoch 10 with training loss: 0.6408590078353882
Epoch 11 with training loss: 0.6371765732765198
Epoch 12 with training loss: 0.6339010000228882
Epoch 13 with training loss: 0.6310127377510071
Epoch 14 with training loss: 0.6284437775611877
Epoch 15 with training loss: 0.6261173486709595
Epoch 16 with training loss: 0.6240249276161194
Epoch 17 with training loss: 0.6221157908439636
Epoch 18 with training loss: 0.6203364729881287
Epoch 19 with training loss: 0.6187050938606262
Epoch 20 with training loss: 0.61718

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:31: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:32: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


Epoch 25 with training loss: 0.610903799533844
Epoch 26 with training loss: 0.6098729372024536
Epoch 27 with training loss: 0.6089073419570923
Epoch 28 with training loss: 0.6079987287521362
Epoch 29 with training loss: 0.6071435809135437
Epoch 30 with training loss: 0.6063304543495178
Epoch 31 with training loss: 0.6055570840835571
Epoch 32 with training loss: 0.6048153042793274
Epoch 33 with training loss: 0.6041056513786316
Epoch 34 with training loss: 0.6034263968467712
Epoch 35 with training loss: 0.6027810573577881
Epoch 36 with training loss: 0.6021671891212463
Epoch 37 with training loss: 0.6015771627426147
Epoch 38 with training loss: 0.601013720035553
Epoch 39 with training loss: 0.6004734039306641
Epoch 40 with training loss: 0.5999528169631958
Epoch 41 with training loss: 0.5994523167610168
Epoch 42 with training loss: 0.5989693403244019
Epoch 43 with training loss: 0.5985031723976135
Epoch 44 with training loss: 0.5980532765388489
Epoch 45 with training loss: 0.59761840105

In [88]:
def convert_to_thresholds(data):
  return data>0.5

def test_model():
  preds = neural_network(test_features).cpu().data.numpy()
  correct = np.argmax(test_labels.cpu().numpy(), axis=1).astype(np.float32)
  preds = np.argmax(preds, axis=1).astype(np.float32)
  print(np.sum(correct==preds)/len(test_labels))

  # preds = convert_to_thresholds(preds)
  # print("F1 Score: ", f1_score(preds, correct, average="micro"))
  # print("ROC-AUC: ", roc_auc_score(correct, preds))

test_model()

0.47368421052631576


##Tests on simplified 2 state binary classification problem
**Initial results tested with 20 labels:**
F1 Score: 0.53
ROC-AUC: 0.51

**Tested on only the most present 8 labels** - as done with the forecasting pipeline (to check if this is a signal to noise ratio)
F1 Score: 0.558
ROC-AUC: 0.501

So the issue here isn't due to noise - the Longformer is just not able to learn a meaningful enough feature representation that can map to the labels.

Let's try to finetune our Longformer model first on some domain-specific text - reports from the WHO that outline how they plan to defeat Meningitis by 2030. After finetuning on this data, let's try to regenerate our feature representations of our report text and see if that changes our results with the multilabel classifier

**After training a pre-trained model to do language modeling**
F1 Score: 0.5329
ROC-AUC: 0.566

The results did improve but not significantly

In [ ]:
#As input to finetune model - need to tokenize and feed in sentences into longformer
#In order to feed text into the longformer - need to pass in 3 things:
#1. input_ids which are the tokenized sentences (with the correct tags e.g. <s>)
#2. input_masks telling model where to pay attention to - i.e. where to ignore padding etc.
#3. token type ids which tell models about relationship of different sentences (e.g. relevant to question and answering datasets) 
# since our input consists of single sentences, we don't need it however

def tokenize_sentences(sentences):
  input_ids, attention_masks = [], []
  for sentence in sentences:
    #encodings is a dictionary with the encoded sequence and additional information like the attention mask
    encoded_dict = tokenizer.encode_plus(sentence, add_special_tokens=True, max_length=4096, pad_to_max_length=True, return_attention_mask=True)
    input_ids.append(encoded_dict["input_ids"])
    attention_masks.append(encoded_dict["attention_mask"])
  return input_ids, attention_masks

with open("finetune.txt", "r") as f:
  corpus = f.read().strip()
  corpus = corpus.replace('."', '".')
  #split the corpus into an array of sentences - each sentence will be its own item in the array
  sentences = re.split("(?<!\w\.\w.)(?<![A-Z][a-z]\.)(?<=\.|\?)\s", corpus)
  sep = int(0.8*len(sentences))
  #save the last 20% of the data for test
  train_dataset, test_dataset = sentences[:sep], sentences[sep:]

input_ids, attention_masks = tokenize_sentences(train_dataset)
print(len(input_ids)*4096)

train_dataset, test_dataset = map(tokenize_sentences, train_sentences), map(tokenize_sentences, test_sentences)
print(train_dataset)

# config = LongformerConfig(dropout=0.2, attention_dropout=0.2)
# config.output_hidden_states = False
#note we're gonna be using the Longformer model we defined above so we don't need to define a new model here

torch.cuda.empty_cache()